# Técnicas de imputación

En este capítulo se aplican distintas técnicas para reemplazar los valores faltantes.  
El propósito es comparar métodos sencillos y avanzados, evaluando cómo transforman la base de datos.

**Puntos clave:**
- Aplicar imputación simple (media, moda, mediana).
- Implementar técnicas más avanzadas como KNN o MICE.
- Comparar las bases imputadas resultantes.
- Identificar ventajas y limitaciones de cada técnica.

Este paso es esencial para preparar los datos antes de la etapa de evaluación.


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# Cargar dataset (asegurando la ruta desde secciones/)
csv_path = Path.cwd().parent / "base_imputacion_mixta_1000.csv"
df = pd.read_csv(csv_path)

print("Archivo cargado:", csv_path.name, "| Dimensiones:", df.shape)

# Separar por tipo
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))


Archivo cargado: base_imputacion_mixta_1000.csv | Dimensiones: (1000, 12)
Numéricas: 6 | Categóricas: 6


In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# Separar por tipo
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

# 1) Imputador simple
num_si = SimpleImputer(strategy="median")
cat_si = SimpleImputer(strategy="most_frequent")

# 2) KNN para numéricas
num_knn = KNNImputer(n_neighbors=5, weights="distance")

# 3) Iterative/MICE para numéricas
num_mice = IterativeImputer(random_state=42, sample_posterior=True, max_iter=10)

# Codificador para categóricas (si luego entrenaras modelos); para imputar solo, no aplica
ohe = OneHotEncoder(handle_unknown="ignore")

# Pipelines de imputación
pipe_simple = ColumnTransformer([
    ("num", num_si, num_cols),
    ("cat", cat_si, cat_cols),
], remainder="drop")

pipe_knn = ColumnTransformer([
    ("num", num_knn, num_cols),
    ("cat", cat_si, cat_cols),
], remainder="drop")

pipe_mice = ColumnTransformer([
    ("num", num_mice, num_cols),
    ("cat", cat_si, cat_cols),
], remainder="drop")

# Fit-transform (crea data imputada)
X_simple = pd.DataFrame(pipe_simple.fit_transform(df), columns=num_cols+cat_cols)
X_knn    = pd.DataFrame(pipe_knn.fit_transform(df), columns=num_cols+cat_cols)
X_mice   = pd.DataFrame(pipe_mice.fit_transform(df), columns=num_cols+cat_cols)

X_simple.head()


,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil
0,19.0,161.821754,3574.753806,1832.731832,640.465372,119.202995,2024-01-01,F,Medellín,Secundaria,B,Unión libre
1,52.0,167.819566,3163.626815,1676.193764,533.10843,124.457874,2024-01-02,F,Barranquilla,Secundaria,B,Soltero/a
2,38.0,165.756219,2765.672259,1219.535074,491.01691,160.721251,2024-01-03,M,Bogotá,Secundaria,B,Soltero/a
3,57.0,160.64267,4320.397345,1908.324816,599.692595,129.426792,2024-01-04,F,Bogotá,Secundaria,B,Casado/a
4,67.0,151.402909,3669.620507,1887.385697,610.213994,133.916319,2024-01-05,M,Cali,Técnico,B,Soltero/a


### Comentario de la tabla (tras imputación simple)

La tabla muestra las primeras 5 filas del dataset **después de aplicar imputación simple**:

- **Numéricas:** (`edad`, `altura_cm`, `ingresos`, `gasto_mensual`, `puntuacion_credito`, `demanda`)  
  → Los valores faltantes fueron reemplazados por la **mediana** de cada variable.  
  Esto asegura que no se distorsione la distribución con valores extremos (como ocurriría con la media).

- **Categóricas:** (`fecha`, `sexo`, `ciudad`, `nivel_educativo`, `segmento`, `estado_civil`)  
  → Los valores faltantes fueron reemplazados por la **categoría más frecuente** en cada columna.  
  Por ejemplo, si el valor más común en `estado_civil` era *Soltero/a*, los `NaN` se sustituyeron por esa categoría.

 **Lo que observamos en la imagen:**  
- No aparecen valores nulos (`NaN`), ya que fueron imputados.  
- Se preserva la estructura del dataset original: las columnas se mantienen, pero ahora con los huecos rellenados.  
- Las variables numéricas mantienen valores realistas (no hay ceros arbitrarios ni valores fuera de rango).  
- Las categóricas muestran categorías válidas, sin vacíos.




In [8]:
# KNN + Moda para categóricas
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

knn_imputer = KNNImputer(n_neighbors=5, weights="distance")
cat_imputer = SimpleImputer(strategy="most_frequent")

pipe_knn = Pipeline([
    ("ct", ColumnTransformer([
        ("num", knn_imputer, num_cols),
        ("cat", cat_imputer, cat_cols),
    ]))
])

df_knn = pd.DataFrame(pipe_knn.fit_transform(df), columns=num_cols + cat_cols)

print("KNN imputado:", df_knn.shape)
df_knn.head()


KNN imputado: (1000, 12)


,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil
0,19.0,161.821754,3574.753806,1832.731832,640.465372,119.202995,2024-01-01,F,Medellín,Secundaria,B,Unión libre
1,52.0,167.819566,3163.626815,1225.425851,533.10843,124.457874,2024-01-02,F,Barranquilla,Secundaria,B,Soltero/a
2,38.0,165.756219,2765.672259,1219.535074,491.01691,136.427119,2024-01-03,M,Bogotá,Secundaria,B,Soltero/a
3,57.0,160.64267,4320.397345,1908.324816,579.254068,129.426792,2024-01-04,F,Bogotá,Secundaria,B,Casado/a
4,67.0,151.402909,4430.046269,1887.385697,610.213994,133.916319,2024-01-05,M,Cali,Técnico,B,Soltero/a


### Imputación con KNN y Moda
En este apartado aplicamos un esquema de imputación combinado:
- Para variables numéricas: KNNImputer con 5 vecinos ponderados por distancia.
- Para variables categóricas: SimpleImputer con moda.

De esta manera, cada tipo de variable recibe un tratamiento adecuado,
manteniendo la consistencia en los datos imputados.
